In [1]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from tqdm import tqdm

In [2]:
import requests
import pandas as pd
import time
from datetime import datetime, timedelta
from tqdm import tqdm

BASE_URL = "https://www.deribit.com/api/v2"

# 1. 전체 옵션 리스트 수집 (미만기 + 만기 포함)
def get_all_option_instruments(currency="BTC"):
    url_live = f"{BASE_URL}/public/get_instruments?currency={currency}&kind=option&expired=false"
    url_expired = f"{BASE_URL}/public/get_instruments?currency={currency}&kind=option&expired=true"

    instruments = []
    for url in [url_live, url_expired]:
        try:
            res = requests.get(url)
            res.raise_for_status()
            instruments += res.json()['result']
            print(f"[OK] 수집 완료: {url}")
        except Exception as e:
            print(f"[ERROR] 옵션 리스트 수집 실패: {url} | {e}")

    print(f"[INFO] 총 옵션 수집됨: {len(instruments)}개")
    return instruments

# 2. 특정 옵션, 특정 기간 체결 정보 가져오기
def get_trades_for_instrument(instrument_name, start_ts, end_ts):
    url = f"{BASE_URL}/public/get_last_trades_by_instrument_and_time"
    params = {
        "instrument_name": instrument_name,
        "start_timestamp": start_ts,
        "end_timestamp": end_ts,
        "count": 1000,
        "include_old": True
    }
    try:
        res = requests.get(url, params=params)
        res.raise_for_status()
        trades = res.json().get("result", {}).get("trades", [])
        print(f"[{instrument_name}] {len(trades)}건 수집됨")
        return trades
    except Exception as e:
        print(f"[ERROR] {instrument_name} | {start_ts} ~ {end_ts} | {e}")
        return []

# 3. 유의미한 옵션만 필터링 (최근 n일간 거래 발생 기준)
def filter_traded_instruments(instruments, days_back=30):
    today = datetime.utcnow()
    start = today - timedelta(days=days_back)
    traded = []
    
    for inst in tqdm(instruments, desc="거래 필터링 중"):
        trades = get_trades_for_instrument(
            inst["instrument_name"],
            int(start.timestamp() * 1000),
            int(today.timestamp() * 1000)
        )
        if trades:
            traded.append(inst)
            print(f"[KEEP] 거래 감지: {inst['instrument_name']} ({len(trades)}건)")
        else:
            print(f"[SKIP] 거래 없음: {inst['instrument_name']}")
    
    return traded

# 4. 개별 옵션의 전체 체결 내역 수집
def collect_all_trades(instrument, start_date, end_date):
    data = []
    curr_date = start_date
    name = instrument['instrument_name']

    print(f"\n[START] 옵션 수집 시작: {name}")
    while curr_date < end_date:
        try:
            start_ts = int(curr_date.timestamp() * 1000)
            end_ts = int((curr_date + timedelta(days=1)).timestamp() * 1000)
            print(f"[{name}] 날짜: {curr_date.date()} → 수집 중...")

            trades = get_trades_for_instrument(name, start_ts, end_ts)

            for t in trades:
                data.append({
                    "instrument_name": name,
                    "expiration": datetime.utcfromtimestamp(instrument["expiration_timestamp"] / 1000),
                    "strike": instrument["strike"],
                    "option_type": instrument["option_type"],
                    "price": t["price"],
                    "amount": t["amount"],
                    "direction": t["direction"],
                    "timestamp": datetime.utcfromtimestamp(t["timestamp"] / 1000)
                })

            curr_date += timedelta(days=1)
            time.sleep(0.2)  # Rate limit 대응

        except Exception as e:
            print(f"[FAIL] {name} @ {curr_date.date()} | {e}")
            curr_date += timedelta(days=1)
            continue

    print(f"[END] 수집 완료: {name} ({len(data)}건)")
    return data

# 5. 전체 파이프라인 실행 (필터링 후 수집)
def run_filtered_collection(start_date: str = "2024-05-01", days_back=30):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.utcnow()

    print(f"\n[STEP 1] 옵션 리스트 수집")
    all_instruments = get_all_option_instruments()

    print(f"\n[STEP 2] 최근 {days_back}일 거래된 옵션만 필터링")
    traded_instruments = filter_traded_instruments(all_instruments, days_back=days_back)

    print(f"\n[STEP 3] 총 거래된 옵션 수: {len(traded_instruments)}개 → 트랜잭션 수집 시작")

    all_data = []
    for inst in tqdm(traded_instruments, desc="옵션 체결 수집 중"):
        trades = collect_all_trades(inst, start_date, end_date)
        all_data.extend(trades)

    df = pd.DataFrame(all_data)
    df.to_csv("deribit_traded_option_trades.csv", index=False)
    print(f"\n[SAVE] 총 {len(df)}건 저장 완료 → deribit_traded_option_trades.csv")


In [3]:
run_filtered_collection()


[STEP 1] 옵션 리스트 수집
[OK] 수집 완료: https://www.deribit.com/api/v2/public/get_instruments?currency=BTC&kind=option&expired=false
[OK] 수집 완료: https://www.deribit.com/api/v2/public/get_instruments?currency=BTC&kind=option&expired=true
[INFO] 총 옵션 수집됨: 762개

[STEP 2] 최근 30일 거래된 옵션만 필터링


거래 필터링 중:   0%|          | 1/762 [00:00<11:06,  1.14it/s]

[BTC-7MAY25-84000-C] 0건 수집됨
[SKIP] 거래 없음: BTC-7MAY25-84000-C


거래 필터링 중:   0%|          | 2/762 [00:01<11:47,  1.07it/s]

[BTC-7MAY25-84000-P] 3건 수집됨
[KEEP] 거래 감지: BTC-7MAY25-84000-P (3건)


...

[END] 수집 완료: BTC-9MAY25-97000-P (102건)

[START] 옵션 수집 시작: BTC-9MAY25-98000-C
[BTC-9MAY25-98000-C] 날짜: 2024-05-01 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-02 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-03 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-04 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-05 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-06 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-07 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-08 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-09 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-10 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-11 → 수집 중...
[BTC-9MAY25-98000-C] 0건 수집됨
[BTC-9MAY25-98000-C] 날짜: 2024-05-12 → 수집 중...
[ERROR] BTC-9MAY25-98000-C | 1715439600000 ~ 1715526000000 | ('

옵션 체결 수집 중:  24%|██▎       | 81/344 [23:13:40<530:48:34, 7265.83s/it]

[END] 수집 완료: BTC-9MAY25-98000-C (0건)

[START] 옵션 수집 시작: BTC-9MAY25-98000-P
[BTC-9MAY25-98000-P] 날짜: 2024-05-01 → 수집 중...
[ERROR] BTC-9MAY25-98000-P | 1714489200000 ~ 1714575600000 | HTTPSConnectionPool(host='www.deribit.com', port=443): Max retries exceeded with url: /api/v2/public/get_last_trades_by_instrument_and_time?instrument_name=BTC-9MAY25-98000-P&start_timestamp=1714489200000&end_timestamp=1714575600000&count=1000&include_old=True (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x145450e20>: Failed to resolve 'www.deribit.com' ([Errno 8] nodename nor servname provided, or not known)"))
[BTC-9MAY25-98000-P] 날짜: 2024-05-02 → 수집 중...
[ERROR] BTC-9MAY25-98000-P | 1714575600000 ~ 1714662000000 | HTTPSConnectionPool(host='www.deribit.com', port=443): Max retries exceeded with url: /api/v2/public/get_last_trades_by_instrument_and_time?instrument_name=BTC-9MAY25-98000-P&start_timestamp=1714575600000&end_timestamp=1714662000000&count=1000&include_old=True (

옵션 체결 수집 중:  24%|██▎       | 81/344 [64:30:29<209:27:08, 2867.03s/it]


KeyboardInterrupt: 